# PyJAMAS Quickie-Net notebook

Use the following folder structure:

train/

	train_folder_name_1/
		image/
			train_image_name_1.tif
		mask/
			train_image_name_1.tif
            prev_mask/
                    train_image_name_1.tif

	.
	.
	.

	train_folder_name_n/
		image/
			train_image_name_n.tif
		mask/
			train_image_name_n.tif
            prev_mask/
                    train_image_mask_n.tif

If using Google Colab, upload the data to Google Drive and mount your drive to access the data.

Disable the PyJAMAS user interface:

In [ ]:
import os
os.environ['QT_QPA_PLATFORM'] = 'offscreen'
os.environ['QT_QPA_PLATFORM_PLUGIN_PATH'] = ''

Install PyJAMAS:

In [ ]:
!pip install pyjamas-rfglab

Create a PyJAMAS instance:

In [ ]:
from pyjamas import PyJAMAS
a = PyJAMAS()

And the folder in your Google Drive in which your data are stored:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Parameters for training:

In [ ]:
parameters = {
    'positive_training_folder': {{positive_training_folder}},
    'save_folder': {{save_folder}},
    'train_image_size': {{train_image_size}},
    'step_sz': {{step_sz}},
    'learning_rate': {{learning_rate}},
    'mini_batch_size': {{mini_batch_size}},
    'epochs': {{epochs}},
    'validation_split': {{validation_split}},
    'erosion_width': {{erosion_width}},
    'concatenation_level': {{concatenation_level}},
    'early_stopper': {{early_stopper}},
    'lr_scheduler': {{lr_scheduler}},
    'model_checkpoint': {{model_checkpoint}},
    'logging': {{logging}},
}

Create and train a QuickieNet:

In [ ]:
a.classifiers.cbCreateQuickieNet(parameters=parameters, wait_for_thread=False) # If you want to block the notebook during training, use wait_for_thread=True.

Save QuickieNet to use with PyJAMAS:

In [ ]:
classifier_filepath = os.path.join(parameters.get('save_folder'), 'quickienet' + PyJAMAS.classifier_extension)

a.io.cbSaveClassifier(classifier_filepath)

If you are using Google Colab, you can download the classifier file and load it locally in PyJAMAS.

Alternatively, you can apply the classifier on Google Colab. For example, if you have an image in /content/drive/test_images/test.tif:

In [ ]:
filepath = '/content/drive/test_images/test.tif'
annotations = '/content/drive/test_images/annotations_tp1.pjs'  # QuickieNets need a prompt, e.g. annotations for the object to be segmented in the first time point of an image sequence.

a.io.cbLoadTimeSeries(filepath)  # Load image into PyJAMAS.
# a.io.cbLoadClassifier(os.path.join(parameters.get('save_folder'), classifier_filename))  # Load classifier (this is not necessary if you just trained your classifier!).
a.io.cbLoadAnnotations([annotations])
a.classifiers.cbApplyClassifier(2, a.n_frames, wait_for_thread=True) # Apply classifier to all time points. If you do not want to block the notebook during inference, use wait_for_thread=False ... but keep in mind that the next line will run before this one finishes!
a.io.cbSaveAnnotations(filepath[:-4] + PyJAMAS.data_extension)  # Save the annotations in the same folder with the image.